In [1]:
import Auto_paint_self

In [2]:
CHM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CHM_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
CMO_RPH,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\CMORPH_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
ERA5_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\ERA5_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GPM_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GPM_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
GSMAP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\GSMAP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
MSWEP_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\MSWEP_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')
PERSIANN_CDR_PRE,lon,lat,levels,latlow,lattop,lonleft,lonright,times=Auto_paint_self.open_data_nc('one',r'E:\Multi-Sources_Precipitation_NC_2001_2022\PERSIANN_CDR_PRE_China_2001_2022_2.nc','Pre','yes','time','2001-01-01','2022-12-31','yes','lon','yes','lat',18.0,53.5,73.5,135.0,0.25,0.25,'no','no',None,None,changeresolution=1,timespace=1,ifchange_west_east='no',ifinterpolate='no')

In [3]:
import numpy as np
vx=np.zeros((CMO_RPH.shape[0],CMO_RPH.shape[1],CMO_RPH.shape[2],6))
vy=np.zeros((CHM_PRE.shape[0],CHM_PRE.shape[1],CHM_PRE.shape[2]),dtype='int32')
CHM_PRE=np.nan_to_num(CHM_PRE,nan=0)
vx[:,:,:,0]=CMO_RPH
vx[:,:,:,1]=ERA5_PRE
vx[:,:,:,2]=GPM_PRE
vx[:,:,:,3]=GSMAP_PRE
vx[:,:,:,4]=MSWEP_PRE
vx[:,:,:,5]=PERSIANN_CDR_PRE
vx=np.nan_to_num(vx,nan=0)
for i in range(CHM_PRE.shape[1]):
    for j in range(CHM_PRE.shape[2]):
        vy[np.array(CHM_PRE[:,i,j])<0.1,i,j]=0
        vy[np.array(CHM_PRE[:,i,j])>=0.1,i,j]=1
vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)
vxs=np.zeros((vx.shape[0]-5,5,vx.shape[1],vx.shape[2],vx.shape[3]))
vys=vy[5:,:,:]
for i in range(vx.shape[0]-5):
    vxs[i,:,:,:,:]=vx[i:i+5,:,:,:]
vxs=(vxs.transpose(0,2,3,1,4))
vxs=vxs.reshape(vxs.shape[0]*vxs.shape[1]*vxs.shape[2],vxs.shape[3],vxs.shape[4])
vys=vys.reshape(vys.shape[0]*vys.shape[1]*vys.shape[2])
print(vxs.shape,vys.shape)

C:\Users\TBYC\AppData\Local\Temp\ipykernel_83108\1209558780.py:16: RuntimeWarning: invalid value encountered in divide
  vx=(vx-np.nanmean(vx,axis=0))/np.nanstd(vx,axis=0)


(283627630, 5, 6) (283627630,)


In [4]:
#Transformer网络
def Auto_Transformer(vy,vx,timestep,model_list,test_size=0.25,task_mode='regression',if_best_mode='no',modelpath=None,encoder_deep=1,num_heads=2,key_dim=2,ifdropout='no',trans_dropout_rate=0.0,trans_units=64,trans_activation='sigmoid',embedding_num=None,if_weight_initialize='no',weight_initialize_method='TruncatedNormal',weight_initialize_parameter1=0.00,weight_initialize_parameter2=0.05,if_print_model='yes',loss_function='default',optimizer='SGD',metrics='default',learning_rate=0.01,epochs=2000,batch_size=20,ifrandom_split='yes',ifweight='yes',ifmute='no',ifsave='no',savepath=None,device='cpu'):
    import tensorflow as tf
    if device=='gpu':
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            try:
                # 设置只使用 GPU 0
                tf.config.set_visible_devices(gpus[0], 'GPU')
                # 设置 GPU 0 的内存动态增长
                tf.config.experimental.set_memory_growth(gpus[0], True)
            except RuntimeError as e:
                print(e)
    from keras.models import Sequential,Model
    from keras.layers.core import Activation,Dropout,Dense
    from keras.layers import Input,BatchNormalization,LayerNormalization,Embedding,Add,MultiHeadAttention,Flatten
    from keras.initializers import TruncatedNormal,RandomNormal,RandomUniform
    from sklearn.model_selection import train_test_split
    import numpy as np
    from tensorflow.keras.optimizers import SGD,Adam
    import keras
    from scipy.stats import pearsonr
    import os
    from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
    from keras.models import load_model
    import sklearn
    import copy
    
    if embedding_num==None:
        embedding_num=vx.shape[1]+1
    if task_mode=='regression':
        if loss_function=='default' or loss_function=='MeanSquaredError':
            loss=tf.keras.losses.MeanSquaredError()
        elif loss_function=='MeanSquaredError':
            loss=tf.keras.losses.MeanSquaredError()
        elif loss_function=='MeanAbsoluteError':
            loss=tf.keras.losses.MeanAbsoluteError()
        elif loss_function=='MeanAbsolutePercentageError':
            loss=tf.keras.losses.MeanAbsolutePercentageError()
        elif loss_function=='MeanSquaredLogarithmicError':
            loss=tf.keras.losses.MeanSquaredLogarithmicError()
        elif loss_function=='CosineSimilarity':
            loss=tf.keras.losses.CosineSimilarity()
        elif loss_function=='Huber':
            loss=tf.keras.losses.Huber()
        elif loss_function=='LogCosh':
            loss=tf.keras.losses.LogCosh()
        elif loss_function=='Pearsonr':
            def loss_pearsonr(y_true,y_pred):
                import tensorflow as tf
                y_true_mean=tf.reduce_mean(y_true,axis=0)
                y_pred_mean=tf.reduce_mean(y_pred,axis=0)
                cov=tf.reduce_sum((y_true-y_true_mean)*(y_pred-y_pred_mean),axis=0)
                y_true_v=tf.reduce_sum(tf.square((y_true-y_true_mean)),axis=0)
                y_pred_v=tf.reduce_sum(tf.square((y_pred-y_pred_mean)),axis=0)
                y_true_v=tf.sqrt(y_true_v)
                y_pred_v=tf.sqrt(y_pred_v)
                pearson=cov/(y_true_v*y_pred_v)
                return (1-pearson)**1.5
            loss=loss_pearsonr
        if metrics=='default' or metrics=='MeanSquaredError':
            metric=tf.keras.metrics.MeanSquaredError()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='MeanAbsolutePercentageError':
            metric=tf.keras.metrics.MeanAbsolutePercentageError()
        elif metrics=='MeanSquaredLogarithmicError':
            metric=tf.keras.metrics.MeanSquaredLogarithmicError()
        elif metrics=='CosineSimilarity':
            metric=tf.keras.metrics.CosineSimilarity()
        elif metrics=='LogCoshError':
            metric=tf.keras.metrics.LogCoshError()
        elif metrics=='Pearsonr':
            def metrics_pearsonr(y_true,y_pred):
                import tensorflow as tf
                y_true_mean=tf.reduce_mean(y_true,axis=0)
                y_pred_mean=tf.reduce_mean(y_pred,axis=0)
                cov=tf.reduce_sum((y_true-y_true_mean)*(y_pred-y_pred_mean),axis=0)
                y_true_v=tf.reduce_sum(tf.square((y_true-y_true_mean)),axis=0)
                y_pred_v=tf.reduce_sum(tf.square((y_pred-y_pred_mean)),axis=0)
                y_true_v=tf.sqrt(y_true_v)
                y_pred_v=tf.sqrt(y_pred_v)
                pearson=cov/(y_true_v*y_pred_v)
                return (1-pearson)**1.5
            metric=metrics_pearsonr
    elif task_mode=='binary_classify':
        if loss_function=='default' or loss_function=='BinaryCrossentropy':
            loss=tf.keras.losses.BinaryCrossentropy()
        elif loss_function=='CategoricalCrossentropy':
            loss=tf.keras.losses.CategoricalCrossentropy()
        elif loss_function=='SparseCategoricalCrossentropy':
            loss=tf.keras.losses.SparseCategoricalCrossentropy()
        elif loss_function=='Poisson':
            loss=tf.keras.losses.Poisson()
        elif loss_function=='KLDivergence':
            loss=tf.keras.losses.KLDivergence()
        if metrics=='default' or metrics=='BinaryAccuracy':
            metric=tf.keras.metrics.BinaryAccuracy()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='TopKCategoricalAccuracy':
            metric=tf.keras.metrics.TopKCategoricalAccuracy()
        elif metrics=='SparseTopKCategoricalAccuracy':
            metric=tf.keras.metrics.SparseTopKCategoricalAccuracy()
        elif metrics=='BinaryCrossentropy':
            metric=tf.keras.metrics.BinaryCrossentropy()
        elif metrics=='CategoricalCrossentropy':
            metric=tf.keras.metrics.CategoricalCrossentropy()
        elif metrics=='SparseCategoricalCrossentropy':
            metric=tf.keras.metrics.SparseCategoricalCrossentropy()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='KLDivergence':
            metric=tf.keras.metrics.KLDivergence()
        elif metrics=='Poisson':
            metric=tf.keras.metrics.Poisson()
        elif metrics=='AUC':
            metric=tf.keras.metrics.AUC()
        elif metrics=='Precision':
            metric=tf.keras.metrics.Precision()
        elif metrics=='Recall':
            metric=tf.keras.metrics.Recall()
        elif metrics=='TruePositives':
            metric=tf.keras.metrics.TruePositives()
        elif metrics=='TrueNegatives':
            metric=tf.keras.metrics.TrueNegatives()
        elif metrics=='FalsePositives':
            metric=tf.keras.metrics.FalsePositives()
        elif metrics=='FalseNegatives':
            metric=tf.keras.metrics.FalseNegatives()
        elif metrics=='PrecisionAtRecall':
            metric=tf.keras.metrics.PrecisionAtRecall()
        elif metrics=='SensitivityAtSpecificity':
            metric=tf.keras.metrics.SensitivityAtSpecificity()
        elif metrics=='SpecificityAtSensitivity':
            metric=tf.keras.metrics.SpecificityAtSensitivity()
    elif task_mode=='multi_classify':
        if loss_function=='default' or loss_function=='CategoricalCrossentropy':
            loss=tf.keras.losses.CategoricalCrossentropy()
        elif loss_function=='SparseCategoricalCrossentropy':
            loss=tf.keras.losses.SparseCategoricalCrossentropy()
        elif loss_function=='Poisson':
            loss=tf.keras.losses.Poisson()
        elif loss_function=='KLDivergence':
            loss=tf.keras.losses.KLDivergence()
        if metrics=='default' or metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='MeanAbsoluteError':
            metric=tf.keras.metrics.MeanAbsoluteError()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='TopKCategoricalAccuracy':
            metric=tf.keras.metrics.TopKCategoricalAccuracy()
        elif metrics=='SparseTopKCategoricalAccuracy':
            metric=tf.keras.metrics.SparseTopKCategoricalAccuracy()
        elif metrics=='CategoricalCrossentropy':
            metric=tf.keras.metrics.CategoricalCrossentropy()
        elif metrics=='SparseCategoricalCrossentropy':
            metric=tf.keras.metrics.SparseCategoricalCrossentropy()
        elif metrics=='Accuracy':
            metric=tf.keras.metrics.Accuracy()
        elif metrics=='CategoricalAccuracy':
            metric=tf.keras.metrics.CategoricalAccuracy()
        elif metrics=='SparseCategoricalAccuracy':
            metric=tf.keras.metrics.SparseCategoricalAccuracy()
        elif metrics=='KLDivergence':
            metric=tf.keras.metrics.KLDivergence()
        elif metrics=='Poisson':
            metric=tf.keras.metrics.Poisson()
    weights=0
    model=0
    if vy.ndim==1:
        vy=vy.reshape(vy.shape[0],1)
    if device=='cpu':
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
        with tf.device('/cpu:0'):
            if ifrandom_split=='yes':
                trainy,testy,trainx,testx = train_test_split(vy,vx,test_size=test_size,random_state=25)
            else:
                index=int((1-test_size)*vy.shape[0])
                trainy=vy[:index]
                testy=vy[index:]
                trainx=vx[:index,:,:]
                testx=vx[index:,:,:]
            train_position=np.zeros((trainx.shape[0],trainx.shape[1]))
            test_position=np.zeros((testx.shape[0],testx.shape[1]))
            for i in range(trainx.shape[0]):
                train_position[i,:]=np.arange(0,timestep,1)
            for i in range(testx.shape[0]):
                test_position[i,:]=np.arange(0,timestep,1)
            def train_data_generator(x, position, y, batch_size):
                num_samples = x.shape[0]
                while True:
                    indices = np.arange(num_samples)
                    np.random.shuffle(indices)
                    for start in range(0, num_samples, batch_size):
                        end = min(start + batch_size, num_samples)
                        batch_indices = indices[start:end]
                        
                        x_batch = x[batch_indices]  # 第一个输入特征
                        position_batch = position[batch_indices]   # 第二个输入特征
                        y_batch = y[batch_indices]      # 标签
                        
                        yield ({"input_1": x_batch, "input_2": position_batch}, y_batch)
            def test_data_generator(x, position, batch_size):
                num_samples = x.shape[0]
                while True:
                    indices = np.arange(num_samples)
                    np.random.shuffle(indices)
                    for start in range(0, num_samples, batch_size):
                        end = min(start + batch_size, num_samples)
                        batch_indices = indices[start:end]
                        
                        x_batch = x[batch_indices]  # 第一个输入特征
                        position_batch = position[batch_indices]
                        
                        yield ({"input_1": x_batch, "input_2": position_batch})
            if if_best_mode=='no':
                inputshape1=(None,timestep,trainx.shape[2])
                inputshape2=(None,timestep)
                inputs1=Input(shape=(timestep,trainx.shape[2]))
                inputs2=Input(shape=(timestep))
                for i in range(len(model_list)):
                    if model_list[i][0] == 'transformer':
                        position_embedding=Embedding(embedding_num,trainx.shape[2],input_length=timestep,input_shape=inputshape2)(inputs2)
                        add=Add(input_shape=inputshape1)([inputs1,position_embedding])
                        for j in range(encoder_deep):
                            if j ==0:
                                exec('en_multihead'+str(j+1)+'=MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,dropout=trans_dropout_rate,attention_axes=1)(add,add,add)')
                                exec('en_add'+str(j+1)+'=Add()([add,en_multihead'+str(j+1)+'])')
                            else:
                                exec('en_multihead'+str(j+1)+'=MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,dropout=trans_dropout_rate,attention_axes=1)(en_layernormalization'+str(j)+',en_layernormalization'+str(j)+',en_layernormalization'+str(j)+')')
                                exec('en_add'+str(j+1)+'=Add()([en_layernormalization'+str(j)+',en_multihead'+str(j+1)+'])')
                            exec('en_layernormalization'+str(j+1)+'=LayerNormalization()(en_add'+str(j+1)+')')
                            if ifdropout=='yes':
                                exec('en_dropout'+str(j+1)+'=Dropout(trans_dropout_rate)(en_layernormalization'+str(j+1)+')')
                                exec('en_fc'+str(j+1)+'=Dense(trans_units,activation=trans_activation)(en_dropout'+str(j+1)+')')
                                exec('en_fc'+str(j+1)+'=Dense(trainx.shape[2],activation=trans_activation)(en_fc'+str(j+1)+')')
                            else:
                                exec('en_fc'+str(j+1)+'=Dense(trans_units,activation=trans_activation)(en_layernormalization'+str(j+1)+')')
                                exec('en_fc'+str(j+1)+'=Dense(trainx.shape[2],activation=trans_activation)(en_fc'+str(j+1)+')')
                            exec('en_add'+str(j+1)+'=Add()([en_fc'+str(j+1)+',en_layernormalization'+str(j+1)+'])')
                            exec('en_layernormalization'+str(j+1)+'=LayerNormalization()(en_add'+str(j+1)+')')
                        exec('en_fla=Flatten()(en_layernormalization'+str(j+1)+')')
                    elif model_list[i][0] == 'batchnormalization':
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                outputs=eval('BatchNormalization(axis=-1)(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                outputs=eval('BatchNormalization(axis=-1)(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                outputs=eval('BatchNormalization(axis=-1)(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                outputs=eval('BatchNormalization(axis=-1)(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                outputs=eval('BatchNormalization(axis=-1)(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                outputs=eval('BatchNormalization(axis=-1)(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(fla'+str(i)+')')
                    elif model_list[i][0] == 'layernormalization':
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                outputs=eval('LayerNormalization(axis=-1)(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                outputs=eval('LayerNormalization(axis=-1)(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                outputs=eval('LayerNormalization(axis=-1)(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                outputs=eval('LayerNormalization(axis=-1)(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                outputs=eval('LayerNormalization(axis=-1)(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                outputs=eval('LayerNormalization(axis=-1)(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(fla'+str(i)+')')
                    elif model_list[i][0] == 'activation':
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                outputs=eval('Activation(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                outputs=eval('Activation(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                outputs=eval('Activation(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                outputs=eval('Activation(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                outputs=eval('Activation(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                outputs=eval('Activation(model_list[i][1])(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                exec('act'+str(i+1)+'=Activation(model_list[i][1])(fla'+str(i)+')')
                    elif model_list[i][0] == 'flatten':
                        if model_list[i-1][0]=='transformer':
                            exec('fla'+str(i+1)+'=Flatten()(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            exec('fla'+str(i+1)+'=Flatten()(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            exec('fla'+str(i+1)+'=Flatten()(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout':
                            exec('fla'+str(i+1)+'=Flatten()(dropout'+str(i)+')')
                    elif model_list[i][0] =='fc':
                        if if_weight_initialize=='no':
                            if i==len(model_list)-1:
                                if model_list[i-1][0]=='transformer':
                                    outputs=eval('Dense(model_list[i][1])(en_fla)')
                                elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                    outputs=eval('Dense(model_list[i][1])(norm'+str(i)+')')
                                elif model_list[i-1][0]=='activation':
                                    outputs=eval('Dense(model_list[i][1])(act'+str(i)+')')
                                elif model_list[i-1][0]=='dropout':
                                    outputs=eval('Dense(model_list[i][1])(drop'+str(i)+')')
                                elif model_list[i-1][0]=='fc':
                                    outputs=eval('Dense(model_list[i][1])(fc'+str(i)+')')
                                elif model_list[i-1][0]=='flatten':
                                    outputs=eval('Dense(model_list[i][1])(fla'+str(i)+')')
                            else:
                                if model_list[i-1][0]=='transformer':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(en_fla)')
                                elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(norm'+str(i)+')')
                                elif model_list[i-1][0]=='activation':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(act'+str(i)+')')
                                elif model_list[i-1][0]=='dropout':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(drop'+str(i)+')')
                                elif model_list[i-1][0]=='fc':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(fc'+str(i)+')')
                                elif model_list[i-1][0]=='flatten':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1])(fla'+str(i)+')')
                        else:
                            if i==len(model_list)-1:
                                if model_list[i-1][0]=='transformer':
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(en_fla)')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif model_list[i-1][0]=='activation':
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(act'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                elif model_list[i-1][0]=='dropout':
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(drop'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif model_list[i-1][0]=='fc':   
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fc'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif model_list[i-1][0]=='flatten':
                                    if weight_initialize_method=='RandomNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fla'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                            else:
                                if model_list[i-1][0]=='transformer':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(en_fla)')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif model_list[i-1][0]=='activation':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(act'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                elif model_list[i-1][0]=='dropout':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(drop'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif model_list[i-1][0]=='fc':   
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fc'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif model_list[i-1][0]=='flatten':
                                    if weight_initialize_method=='RandomNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                                    elif weight_initialize_method=='RandomUniform':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fla'+str(i)+')')
                                    elif weight_initialize_method=='TruncatedNormal':
                                        exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                    elif model_list[i][0] == 'dropout':
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                outputs=eval('Dropout(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                outputs=eval('Dropout(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                outputs=eval('Dropout(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                outputs=eval('Dropout(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                outputs=eval('Dropout(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                outputs=eval('Dropout(model_list[i][1])(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout' :
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                exec('act'+str(i+1)+'=Dropout(model_list[i][1])(fla'+str(i)+')')
                model=eval('Model(inputs=[inputs1,inputs2], outputs=outputs)')
                if optimizer == 'SGD':
                    opt = SGD(lr = learning_rate)
                elif optimizer == 'Adam':
                    opt = Adam(lr = learning_rate)
                model.compile(loss=loss,optimizer=opt,metrics=[metric])
            elif if_best_mode=='yes' or if_best_mode=='load':
                model=load_model(modelpath)
            if if_print_model=='yes':
                print(model.summary())
            trainx=np.nan_to_num(trainx,nan=0)
            testx=np.nan_to_num(testx,nan=0)
            if epochs!=0:
                H = model.fit_generator(train_data_generator(trainx,train_position,trainy,batch_size),steps_per_epoch=(len(trainx) // batch_size+(1 if len(trainx) % batch_size != 0 else 0)),validation_data=train_data_generator(testx,test_position,testy,batch_size),validation_steps=(len(testx) // batch_size+(1 if len(testx) % batch_size != 0 else 0)),epochs = epochs)
            predicty = model.predict(test_data_generator(testx,test_position,batch_size),steps=(len(testx) // batch_size+(1 if len(testx) % batch_size != 0 else 0)))
            predicty = np.nan_to_num(predicty,nan=0)
            if task_mode=='regression':
                r=np.zeros((testy.shape[1]))
                p=np.zeros((testy.shape[1]))
                for i in range(testy.shape[1]):
                    r[i],p[i] = pearsonr(predicty[:,i],testy[:,i])
                    r=np.nan_to_num(r,nan=0)
            elif task_mode=='binary_classify':
                accuracy=np.zeros((testy.shape[1]))
                recall=np.zeros((testy.shape[1]))
                precision=np.zeros((testy.shape[1]))
                f1=np.zeros((testy.shape[1]))
                for i in range(predicty.shape[1]):
                    predicty[:,i]=[int(round(predicty[j,i],0)) for j in range(predicty.shape[0])]
                r=np.zeros((testy.shape[1]))
                for i in range(testy.shape[1]):
                    if metrics=='Recall':
                        r[i]=recall_score(testy[:,i], predicty[:,i])
                    elif metrics=='Precision':
                        r[i]=precision_score(testy[:,i], predicty[:,i])
                    else:
                        r[i]=accuracy_score(testy[:,i], predicty[:,i])
                    recall[i]=recall_score(testy[:,i], predicty[:,i])
                    precision[i]=precision_score(testy[:,i], predicty[:,i])
                    accuracy[i]=accuracy_score(testy[:,i], predicty[:,i])
                    f1[i]=f1_score(testy[:,i], predicty[:,i])
                p=0
            elif task_mode=='multi_classify':
                for i in range(predicty.shape[1]):
                    predicty[:,i]=[int(round(predicty[j,i],0)) for j in range(predicty.shape[0])]
                r=np.zeros((testy.shape[1]))
                for i in range(testy.shape[1]):
                    r[i]=accuracy_score(testy[:,i], predicty[:,i])
                p=0
            if ifmute == 'no':
                if task_mode=='regression':
                    print('相关系数',np.nanmean(r))
                elif task_mode=='binary_classify':
                    print('召回率+精确率',np.nanmean(f1),'准确率',np.nanmean(accuracy),'召回率',np.nanmean(recall),'精确率',np.nanmean(precision))
                elif task_mode=='multi_classify':
                    print('准确率',np.nanmean(r))
            if ifweight=='yes':
                weights=np.zeros((testy.shape[1],testx.shape[2]))
                weight_more=np.zeros((testy.shape[1],testx.shape[2]))
                for i in range(testy.shape[1]):
                    for j in range(testx.shape[2]):
                        testx_new=copy.deepcopy(testx)
                        weight=[]
                        for k in range(10):
                            per=np.random.permutation(testx.shape[0])
                            testx_shuffle=testx[per,:,j]
                            testx_new[:,:,j]=testx_shuffle
                            predicty_new=np.array(model.predict(test_data_generator(testx_new,test_position,batch_size),steps=(len(testx_new) // batch_size+(1 if len(testx) % batch_size != 0 else 0))))[:,i]
                            if task_mode=='regression':
                                weight.append(sklearn.metrics.mean_squared_error(testy[:,i],predicty_new)-sklearn.metrics.mean_squared_error(testy[:,i],predicty[:,i]))
                            else:
                                weight.append(sklearn.metrics.log_loss(testy[:,i],predicty_new)-sklearn.metrics.log_loss(testy[:,i],predicty[:,i]))
                        weight_more[i,j]=np.nanmean(weight)
                for i in range(testy.shape[1]):
                    for j in range(testx.shape[2]):
                        weights[i,j]=(weight_more[i,j]/np.nansum(weight_more[i,:]))*100
                        print('预报因子',j+1,'对预报值',i+1,'的贡献：',np.array(weights[i,j]),'％')
                    print('\n')
            if ifsave=='yes':
                model.save(savepath)
    elif device=='gpu':
        if ifrandom_split=='yes':
            trainy,testy,trainx,testx = train_test_split(vy,vx,test_size=test_size,random_state=25)
        else:
            index=int((1-test_size)*vy.shape[0])
            trainy=vy[:index]
            testy=vy[index:]
            trainx=vx[:index,:,:]
            testx=vx[index:,:,:]
        train_position=np.zeros((trainx.shape[0],trainx.shape[1]))
        test_position=np.zeros((testx.shape[0],testx.shape[1]))
        for i in range(trainx.shape[0]):
            train_position[i,:]=np.arange(0,timestep,1)
        for i in range(testx.shape[0]):
            test_position[i,:]=np.arange(0,timestep,1)
        def train_data_generator(x, position, y, batch_size):
            num_samples = x.shape[0]
            while True:
                indices = np.arange(num_samples)
                np.random.shuffle(indices)
                for start in range(0, num_samples, batch_size):
                    end = min(start + batch_size, num_samples)
                    batch_indices = indices[start:end]
                    
                    x_batch = x[batch_indices]  # 第一个输入特征
                    position_batch = position[batch_indices]   # 第二个输入特征
                    y_batch = y[batch_indices]      # 标签
                    
                    yield ({"input_1": x_batch, "input_2": position_batch}, y_batch)
        def test_data_generator(x, position, batch_size):
            num_samples = x.shape[0]
            while True:
                indices = np.arange(num_samples)
                np.random.shuffle(indices)
                for start in range(0, num_samples, batch_size):
                    end = min(start + batch_size, num_samples)
                    batch_indices = indices[start:end]
                    
                    x_batch = x[batch_indices]  # 第一个输入特征
                    position_batch = position[batch_indices]
                    
                    yield ({"input_1": x_batch, "input_2": position_batch})
        if if_best_mode=='no':
            inputshape1=(None,timestep,trainx.shape[2])
            inputshape2=(None,timestep)
            inputs1=Input(shape=(timestep,trainx.shape[2]))
            inputs2=Input(shape=(timestep))
            for i in range(len(model_list)):
                if model_list[i][0] == 'transformer':
                    position_embedding=Embedding(embedding_num,trainx.shape[2],input_length=timestep,input_shape=inputshape2)(inputs2)
                    add=Add(input_shape=inputshape1)([inputs1,position_embedding])
                    for j in range(encoder_deep):
                        if j ==0:
                            exec('en_multihead'+str(j+1)+'=MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,dropout=trans_dropout_rate,attention_axes=1)(add,add,add)')
                            exec('en_add'+str(j+1)+'=Add()([add,en_multihead'+str(j+1)+'])')
                        else:
                            exec('en_multihead'+str(j+1)+'=MultiHeadAttention(num_heads=num_heads,key_dim=key_dim,dropout=trans_dropout_rate,attention_axes=1)(en_layernormalization'+str(j)+',en_layernormalization'+str(j)+',en_layernormalization'+str(j)+')')
                            exec('en_add'+str(j+1)+'=Add()([en_layernormalization'+str(j)+',en_multihead'+str(j+1)+'])')
                        exec('en_layernormalization'+str(j+1)+'=LayerNormalization()(en_add'+str(j+1)+')')
                        if ifdropout=='yes':
                            exec('en_dropout'+str(j+1)+'=Dropout(trans_dropout_rate)(en_layernormalization'+str(j+1)+')')
                            exec('en_fc'+str(j+1)+'=Dense(trans_units,activation=trans_activation)(en_dropout'+str(j+1)+')')
                            exec('en_fc'+str(j+1)+'=Dense(trainx.shape[2],activation=trans_activation)(en_fc'+str(j+1)+')')
                        else:
                            exec('en_fc'+str(j+1)+'=Dense(trans_units,activation=trans_activation)(en_layernormalization'+str(j+1)+')')
                            exec('en_fc'+str(j+1)+'=Dense(trainx.shape[2],activation=trans_activation)(en_fc'+str(j+1)+')')
                        exec('en_add'+str(j+1)+'=Add()([en_fc'+str(j+1)+',en_layernormalization'+str(j+1)+'])')
                        exec('en_layernormalization'+str(j+1)+'=LayerNormalization()(en_add'+str(j+1)+')')
                    exec('en_fla=Flatten()(en_layernormalization'+str(j+1)+')')
                elif model_list[i][0] == 'batchnormalization':
                    if i==len(model_list)-1:
                        if model_list[i-1][0]=='transformer':
                            outputs=eval('BatchNormalization(axis=-1)(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            outputs=eval('BatchNormalization(axis=-1)(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            outputs=eval('BatchNormalization(axis=-1)(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            outputs=eval('BatchNormalization(axis=-1)(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            outputs=eval('BatchNormalization(axis=-1)(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            outputs=eval('BatchNormalization(axis=-1)(fla'+str(i)+')')
                    else:
                        if model_list[i-1][0]=='transformer':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            exec('norm'+str(i+1)+'=BatchNormalization(axis=-1)(fla'+str(i)+')')
                elif model_list[i][0] == 'layernormalization':
                    if i==len(model_list)-1:
                        if model_list[i-1][0]=='transformer':
                            outputs=eval('LayerNormalization(axis=-1)(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            outputs=eval('LayerNormalization(axis=-1)(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            outputs=eval('LayerNormalization(axis=-1)(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            outputs=eval('LayerNormalization(axis=-1)(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            outputs=eval('LayerNormalization(axis=-1)(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            outputs=eval('LayerNormalization(axis=-1)(fla'+str(i)+')')
                    else:
                        if model_list[i-1][0]=='transformer':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            exec('norm'+str(i+1)+'=LayerNormalization(axis=-1)(fla'+str(i)+')')
                elif model_list[i][0] == 'activation':
                    if i==len(model_list)-1:
                        if model_list[i-1][0]=='transformer':
                            outputs=eval('Activation(model_list[i][1])(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            outputs=eval('Activation(model_list[i][1])(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            outputs=eval('Activation(model_list[i][1])(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            outputs=eval('Activation(model_list[i][1])(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            outputs=eval('Activation(model_list[i][1])(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            outputs=eval('Activation(model_list[i][1])(fla'+str(i)+')')
                    else:
                        if model_list[i-1][0]=='transformer':
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            exec('act'+str(i+1)+'=Activation(model_list[i][1])(fla'+str(i)+')')
                elif model_list[i][0] == 'flatten':
                    if model_list[i-1][0]=='transformer':
                        exec('fla'+str(i+1)+'=Flatten()(en_fla)')
                    elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                        exec('fla'+str(i+1)+'=Flatten()(norm'+str(i)+')')
                    elif model_list[i-1][0]=='activation':
                        exec('fla'+str(i+1)+'=Flatten()(act'+str(i)+')')
                    elif model_list[i-1][0]=='dropout':
                        exec('fla'+str(i+1)+'=Flatten()(dropout'+str(i)+')')
                elif model_list[i][0] =='fc':
                    if if_weight_initialize=='no':
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                outputs=eval('Dense(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                outputs=eval('Dense(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                outputs=eval('Dense(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout':
                                outputs=eval('Dense(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                outputs=eval('Dense(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                outputs=eval('Dense(model_list[i][1])(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                exec('fc'+str(i+1)+'=Dense(model_list[i][1])(fla'+str(i)+')')
                    else:
                        if i==len(model_list)-1:
                            if model_list[i-1][0]=='transformer':
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(en_fla)')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(act'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout':
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':   
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                if weight_initialize_method=='RandomNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fla'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    outputs=eval('Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                        else:
                            if model_list[i-1][0]=='transformer':
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(en_fla)')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(en_fla)')
                            elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(norm'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(norm'+str(i)+')')
                            elif model_list[i-1][0]=='activation':
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(act'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(act'+str(i)+')')
                            elif model_list[i-1][0]=='dropout':
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(drop'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(drop'+str(i)+')')
                            elif model_list[i-1][0]=='fc':   
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fc'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fc'+str(i)+')')
                            elif model_list[i-1][0]=='flatten':
                                if weight_initialize_method=='RandomNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                                elif weight_initialize_method=='RandomUniform':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = RandomUniform(minval=weight_initialize_parameter1,maxval=weight_initialize_parameter2))(fla'+str(i)+')')
                                elif weight_initialize_method=='TruncatedNormal':
                                    exec('fc'+str(i+1)+'=Dense(model_list[i][1],kernel_initializer = TruncatedNormal(mean=weight_initialize_parameter1,stddev=weight_initialize_parameter2))(fla'+str(i)+')')
                elif model_list[i][0] == 'dropout':
                    if i==len(model_list)-1:
                        if model_list[i-1][0]=='transformer':
                            outputs=eval('Dropout(model_list[i][1])(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            outputs=eval('Dropout(model_list[i][1])(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            outputs=eval('Dropout(model_list[i][1])(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            outputs=eval('Dropout(model_list[i][1])(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            outputs=eval('Dropout(model_list[i][1])(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            outputs=eval('Dropout(model_list[i][1])(fla'+str(i)+')')
                    else:
                        if model_list[i-1][0]=='transformer':
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(en_fla)')
                        elif model_list[i-1][0]=='batchnormalization' or model_list[i-1][0]=='layernormalization':
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(norm'+str(i)+')')
                        elif model_list[i-1][0]=='activation':
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(act'+str(i)+')')
                        elif model_list[i-1][0]=='dropout' :
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(drop'+str(i)+')')
                        elif model_list[i-1][0]=='fc':
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(fc'+str(i)+')')
                        elif model_list[i-1][0]=='flatten':
                            exec('act'+str(i+1)+'=Dropout(model_list[i][1])(fla'+str(i)+')')
            model=eval('Model(inputs=[inputs1,inputs2], outputs=outputs)')
            if optimizer == 'SGD':
                opt = SGD(lr = learning_rate)
            elif optimizer == 'Adam':
                opt = Adam(lr = learning_rate)
            model.compile(loss=loss,optimizer=opt,metrics=[metric])
        elif if_best_mode=='yes' or if_best_mode=='load':
            model=load_model(modelpath)
        if if_print_model=='yes':
            print(model.summary())
        trainx=np.nan_to_num(trainx,nan=0)
        testx=np.nan_to_num(testx,nan=0)
        if epochs!=0:
            H = model.fit_generator(train_data_generator(trainx,train_position,trainy,batch_size),steps_per_epoch=(len(trainx) // batch_size+(1 if len(trainx) % batch_size != 0 else 0)),validation_data=train_data_generator(testx,test_position,testy,batch_size),validation_steps=(len(testx) // batch_size+(1 if len(testx) % batch_size != 0 else 0)),epochs = epochs)
        predicty = model.predict(test_data_generator(testx,test_position,batch_size),steps=(len(testx) // batch_size+(1 if len(testx) % batch_size != 0 else 0)))
        predicty = np.nan_to_num(predicty,nan=0)
        if task_mode=='regression':
            r=np.zeros((testy.shape[1]))
            p=np.zeros((testy.shape[1]))
            for i in range(testy.shape[1]):
                r[i],p[i] = pearsonr(predicty[:,i],testy[:,i])
                r=np.nan_to_num(r,nan=0)
        elif task_mode=='binary_classify':
            accuracy=np.zeros((testy.shape[1]))
            recall=np.zeros((testy.shape[1]))
            precision=np.zeros((testy.shape[1]))
            f1=np.zeros((testy.shape[1]))
            for i in range(predicty.shape[1]):
                predicty[:,i]=[int(round(predicty[j,i],0)) for j in range(predicty.shape[0])]
            r=np.zeros((testy.shape[1]))
            for i in range(testy.shape[1]):
                if metrics=='Recall':
                    r[i]=recall_score(testy[:,i], predicty[:,i])
                elif metrics=='Precision':
                    r[i]=precision_score(testy[:,i], predicty[:,i])
                else:
                    r[i]=accuracy_score(testy[:,i], predicty[:,i])
                recall[i]=recall_score(testy[:,i], predicty[:,i])
                precision[i]=precision_score(testy[:,i], predicty[:,i])
                accuracy[i]=accuracy_score(testy[:,i], predicty[:,i])
                f1[i]=f1_score(testy[:,i], predicty[:,i])
            p=0
        elif task_mode=='multi_classify':
            for i in range(predicty.shape[1]):
                predicty[:,i]=[int(round(predicty[j,i],0)) for j in range(predicty.shape[0])]
            r=np.zeros((testy.shape[1]))
            for i in range(testy.shape[1]):
                r[i]=accuracy_score(testy[:,i], predicty[:,i])
            p=0
        if ifmute == 'no':
            if task_mode=='regression':
                print('相关系数',np.nanmean(r))
            elif task_mode=='binary_classify':
                print('召回率+精确率',np.nanmean(f1),'准确率',np.nanmean(accuracy),'召回率',np.nanmean(recall),'精确率',np.nanmean(precision))
            elif task_mode=='multi_classify':
                print('准确率',np.nanmean(r))
        if ifweight=='yes':
            weights=np.zeros((testy.shape[1],testx.shape[2]))
            weight_more=np.zeros((testy.shape[1],testx.shape[2]))
            for i in range(testy.shape[1]):
                for j in range(testx.shape[2]):
                    testx_new=copy.deepcopy(testx)
                    weight=[]
                    for k in range(10):
                        per=np.random.permutation(testx.shape[0])
                        testx_shuffle=testx[per,:,j]
                        testx_new[:,:,j]=testx_shuffle
                        predicty_new=np.array(model.predict(test_data_generator(testx_new,test_position,batch_size),steps=len((testx_new) // batch_size)+(1 if len(testx_new) % batch_size != 0 else 0)))[:,i]
                        if task_mode=='regression':
                            weight.append(sklearn.metrics.mean_squared_error(testy[:,i],predicty_new)-sklearn.metrics.mean_squared_error(testy[:,i],predicty[:,i]))
                        else:
                            weight.append(sklearn.metrics.log_loss(testy[:,i],predicty_new)-sklearn.metrics.log_loss(testy[:,i],predicty[:,i]))
                    weight_more[i,j]=np.nanmean(weight)
            for i in range(testy.shape[1]):
                for j in range(testx.shape[2]):
                    weights[i,j]=(weight_more[i,j]/np.nansum(weight_more[i,:]))*100
                    print('预报因子',j+1,'对预报值',i+1,'的贡献：',np.array(weights[i,j]),'％')
                print('\n')
        if ifsave=='yes':
            model.save(savepath)
    return model,predicty,testy,r,p,weights

In [5]:
model,predicty,testy,r,p,weights=Auto_Transformer(vys,vxs,5,[['transformer'],['fc',1024],['batchnormalization'],['activation','tanh'],['fc',1],['batchnormalization'],['activation','sigmoid']],test_size=0.2,task_mode='binary_classify',if_best_mode='no',modelpath=None,encoder_deep=2,num_heads=2,key_dim=2,ifdropout='no',trans_dropout_rate=0.0,trans_units=64,trans_activation='sigmoid',embedding_num=None,if_weight_initialize='no',weight_initialize_method='TruncatedNormal',weight_initialize_parameter1=0.00,weight_initialize_parameter2=0.05,if_print_model='yes',loss_function='default',optimizer='Adam',metrics='Accuracy',learning_rate=0.01,epochs=100,batch_size=500000,ifrandom_split='no',ifweight='no',ifmute='no',ifsave='yes',savepath=r'E:/xuanxuan/xuanxuan_transformer_classify',device='gpu')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 5, 6)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 5, 6)         36          ['input_2[0][0]']                
                                                                                                  
 add (Add)                      (None, 5, 6)         0           ['input_1[0][0]',                
                                                                  'embedding[0][0]']          

C:\Users\TBYC\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


None


C:\Users\TBYC\AppData\Local\Temp\ipykernel_83108\4277553423.py:883: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_data_generator(trainx,train_position,trainy,batch_size),steps_per_epoch=(len(trainx) // batch_size+(1 if len(trainx) % batch_size != 0 else 0)),validation_data=train_data_generator(testx,test_position,testy,batch_size),validation_steps=(len(testx) // batch_size+(1 if len(testx) % batch_size != 0 else 0)),epochs = epochs)


Epoch 1/100
454/454 [==============================] - 176s 383ms/step - loss: 0.3064 - accuracy: 0.0000e+00 - val_loss: 0.3241 - val_accuracy: 0.0000e+00
Epoch 2/100
454/454 [==============================] - 170s 375ms/step - loss: 0.2145 - accuracy: 0.0000e+00 - val_loss: 0.4005 - val_accuracy: 0.0000e+00
Epoch 3/100
454/454 [==============================] - 171s 376ms/step - loss: 0.2043 - accuracy: 0.0000e+00 - val_loss: 0.2678 - val_accuracy: 0.0000e+00
Epoch 4/100
454/454 [==============================] - 171s 376ms/step - loss: 0.2006 - accuracy: 0.0000e+00 - val_loss: 0.2108 - val_accuracy: 0.0000e+00
Epoch 5/100
454/454 [==============================] - 170s 376ms/step - loss: 0.1986 - accuracy: 0.0000e+00 - val_loss: 0.2178 - val_accuracy: 0.0000e+00
Epoch 6/100
454/454 [==============================] - 170s 375ms/step - loss: 0.1971 - accuracy: 0.0000e+00 - val_loss: 0.1997 - val_accuracy: 0.0000e+00
Epoch 7/100
454/454 [==============================] - 171s 377ms/step

INFO:tensorflow:Assets written to: E:/xuanxuan/xuanxuan_transformer_classify\assets


INFO:tensorflow:Assets written to: E:/xuanxuan/xuanxuan_transformer_classify\assets
